# Part X: Advanced Topics and Ecosystem

## Chapter 22: Error Handling

Production applications must handle failures gracefully. A robust error handling strategy provides consistent API responses, prevents information leakage, enables debugging through structured logging, and maintains system stability under adverse conditions. This chapter covers FastAPI's exception handling mechanisms, custom error responses, and production-grade logging with correlation tracking.

---

### 22.1 HTTPException: Raising HTTP Errors

FastAPI's `HTTPException` is the standard mechanism for returning error responses with appropriate HTTP status codes. It integrates automatically with OpenAPI documentation and provides structured error responses.

#### HTTPException Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    HTTPException Flow                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Endpoint Code                                                   │
│       │                                                          │
│       ▼                                                          │
│  ┌─────────────────┐    ┌─────────────────┐                   │
│  │  raise          │───▶│  HTTPException  │                   │
│  │  HTTPException( │    │                 │                   │
│  │    status_code= │    │  • status_code │                   │
│  │    404,         │    │  • detail       │                   │
│  │    detail="Not   │    │  • headers      │                   │
│  │    found"       │    │                 │                   │
│  │  )              │    │                 │                   │
│  └─────────────────┘    └────────┬────────┘                   │
│                                  │                             │
│                                  ▼                             │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │              FastAPI Exception Handler                   │    │
│  │                                                         │    │
│  │  Converts to JSON response:                            │    │
│  │  {                                                      │    │
│  │    "detail": "Not found"                                │    │
│  │  }                                                      │    │
│  │                                                         │    │
│  │  HTTP/1.1 404 Not Found                                 │    │
│  │  Content-Type: application/json                         │    │
│  └─────────────────────────────────────────────────────────┘    │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### Basic HTTPException Usage

```python
# error_basic.py - HTTPException patterns
from fastapi import FastAPI, HTTPException, status
from fastapi.responses import JSONResponse

app = FastAPI()

# ═════════════════════════════════════════════════════════════════
# Standard HTTP Exceptions
# ═════════════════════════════════════════════════════════════════

@app.get("/items/{item_id}")
async def read_item(item_id: int):
    """
    Raise 404 when resource not found.
    
    FastAPI automatically converts HTTPException to JSON response.
    """
    if item_id < 0:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Item ID must be positive"
        )
    
    if item_id > 1000:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found"
        )
    
    return {"item_id": item_id}


@app.post("/items/", status_code=status.HTTP_201_CREATED)
async def create_item(item: ItemCreate):
    """Raise 409 on conflict."""
    existing = await get_item_by_name(item.name)
    if existing:
        raise HTTPException(
            status_code=status.HTTP_409_CONFLICT,
            detail=f"Item with name '{item.name}' already exists"
        )
    
    return await save_item(item)


# ═════════════════════════════════════════════════════════════════
# Custom Headers in Errors
# ═════════════════════════════════════════════════════════════════

@app.get("/rate-limited/")
async def rate_limited_endpoint():
    """
    Return 429 with Retry-After header.
    
    Headers help clients implement exponential backoff.
    """
    raise HTTPException(
        status_code=status.HTTP_429_TOO_MANY_REQUESTS,
        detail="Rate limit exceeded",
        headers={"Retry-After": "60"}  # Seconds to wait
    )


# ═════════════════════════════════════════════════════════════════
# Multi-Field Errors (Pydantic-style)
# ═════════════════════════════════════════════════════════════════

@app.post("/users/")
async def create_user(user: UserCreate):
    """
    Return validation-style errors for multiple fields.
    
    While Pydantic handles schema validation, business logic
    errors may need custom validation.
    """
    errors = []
    
    if await user_exists(user.username):
        errors.append({
            "loc": ["body", "username"],
            "msg": "Username already taken",
            "type": "value_error.unique"
        })
    
    if await email_exists(user.email):
        errors.append({
            "loc": ["body", "email"],
            "msg": "Email already registered",
            "type": "value_error.unique"
        })
    
    if errors:
        raise HTTPException(
            status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
            detail=errors  # List of error objects
        )
    
    return await save_user(user)


# ═════════════════════════════════════════════════════════════════
# Security: Prevent Information Leakage
# ═════════════════════════════════════════════════════════════════

@app.post("/auth/login")
async def login(credentials: LoginRequest):
    """
    Return generic error to prevent user enumeration.
    
    Don't reveal whether username or password was wrong.
    """
    user = await get_user(credentials.username)
    
    # Generic error regardless of which check fails
    if not user or not verify_password(credentials.password, user.hashed_password):
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid credentials",
            headers={"WWW-Authenticate": "Bearer"}
        )
    
    if not user.is_active:
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Account disabled"
        )
    
    return {"token": create_token(user)}
```

---

### 22.2 Custom Exception Handlers: Global Error Handling

For production APIs, you need consistent error response formats, centralized logging, and custom handling for specific exception types.

#### Global Exception Handler Implementation

```python
# error_handlers.py - Production error handling
from fastapi import FastAPI, Request, status
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from starlette.exceptions import HTTPException as StarletteHTTPException
from sqlalchemy.exc import SQLAlchemyError, IntegrityError
from jose import JWTError
import logging
import traceback
import uuid
from datetime import datetime
from typing import Any, Dict

logger = logging.getLogger(__name__)

# ═════════════════════════════════════════════════════════════════
# Custom Exception Classes
# ═════════════════════════════════════════════════════════════════

class AppException(Exception):
    """
    Base application exception.
    
    All custom exceptions inherit from this for consistent handling.
    """
    def __init__(
        self,
        message: str,
        status_code: int = 500,
        detail: Dict[str, Any] = None,
        error_code: str = None
    ):
        self.message = message
        self.status_code = status_code
        self.detail = detail or {}
        self.error_code = error_code or f"ERR_{status_code}"
        super().__init__(self.message)


class ResourceNotFoundException(AppException):
    """404 Not Found with specific resource info."""
    def __init__(self, resource: str, resource_id: str):
        super().__init__(
            message=f"{resource} with id '{resource_id}' not found",
            status_code=status.HTTP_404_NOT_FOUND,
            detail={"resource": resource, "id": resource_id},
            error_code="ERR_NOT_FOUND"
        )


class PermissionDeniedException(AppException):
    """403 Forbidden."""
    def __init__(self, action: str = None):
        super().__init__(
            message="Permission denied",
            status_code=status.HTTP_403_FORBIDDEN,
            detail={"action": action},
            error_code="ERR_FORBIDDEN"
        )


class BusinessLogicException(AppException):
    """422 Unprocessable Entity for business rule violations."""
    def __init__(self, message: str, details: Dict = None):
        super().__init__(
            message=message,
            status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
            detail=details,
            error_code="ERR_BUSINESS_RULE"
        )


class ExternalServiceException(AppException):
    """503 Service Unavailable for external API failures."""
    def __init__(self, service: str, original_error: str = None):
        super().__init__(
            message=f"External service '{service}' unavailable",
            status_code=status.HTTP_503_SERVICE_UNAVAILABLE,
            detail={"service": service, "original_error": original_error},
            error_code="ERR_EXTERNAL_SERVICE"
        )


# ═════════════════════════════════════════════════════════════════
# Global Exception Handlers
# ═════════════════════════════════════════════════════════════════

def create_error_response(
    request: Request,
    status_code: int,
    message: str,
    error_code: str = None,
    details: Dict = None,
    exc: Exception = None
) -> JSONResponse:
    """
    Create standardized error response.
    
    All errors follow same format for client consistency.
    """
    error_id = str(uuid.uuid4())
    
    response_body = {
        "error": {
            "id": error_id,
            "code": error_code or f"ERR_{status_code}",
            "message": message,
            "status": status_code,
            "timestamp": datetime.utcnow().isoformat(),
            "path": str(request.url.path),
            "details": details or {}
        }
    }
    
    # Log with correlation ID from request state
    request_id = getattr(request.state, "request_id", "unknown")
    
    log_data = {
        "request_id": request_id,
        "error_id": error_id,
        "status_code": status_code,
        "path": request.url.path,
        "method": request.method,
        "error_code": error_code,
        "message": message
    }
    
    if status_code >= 500:
        logger.error(
            f"Server error: {message}",
            extra=log_data,
            exc_info=exc
        )
    elif status_code >= 400:
        logger.warning(
            f"Client error: {message}",
            extra=log_data
        )
    
    return JSONResponse(
        status_code=status_code,
        content=response_body
    )


def setup_exception_handlers(app: FastAPI):
    """
    Register all exception handlers with FastAPI app.
    
    Call this during app initialization.
    """
    
    # Handler for custom AppException
    @app.exception_handler(AppException)
    async def app_exception_handler(request: Request, exc: AppException):
        return create_error_response(
            request=request,
            status_code=exc.status_code,
            message=exc.message,
            error_code=exc.error_code,
            details=exc.detail,
            exc=exc
        )
    
    # Handler for FastAPI/Starlette HTTPException
    @app.exception_handler(StarletteHTTPException)
    async def http_exception_handler(request: Request, exc: StarletteHTTPException):
        return create_error_response(
            request=request,
            status_code=exc.status_code,
            message=exc.detail,
            details={"headers": dict(exc.headers)} if exc.headers else None
        )
    
    # Handler for Pydantic validation errors
    @app.exception_handler(RequestValidationError)
    async def validation_exception_handler(request: Request, exc: RequestValidationError):
        # Format Pydantic errors into readable structure
        errors = []
        for error in exc.errors():
            errors.append({
                "field": ".".join(str(x) for x in error["loc"]),
                "message": error["msg"],
                "type": error["type"]
            })
        
        return create_error_response(
            request=request,
            status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
            message="Validation error",
            error_code="ERR_VALIDATION",
            details={"errors": errors},
            exc=exc
        )
    
    # Handler for SQLAlchemy database errors
    @app.exception_handler(SQLAlchemyError)
    async def database_exception_handler(request: Request, exc: SQLAlchemyError):
        # Don't expose database details to client
        error_message = "Database error"
        details = {}
        
        if isinstance(exc, IntegrityError):
            error_message = "Data integrity error"
            error_code = "ERR_INTEGRITY"
            # Extract constraint name if possible (PostgreSQL)
            if "unique constraint" in str(exc.orig).lower():
                error_message = "Duplicate entry"
                error_code = "ERR_DUPLICATE"
        else:
            error_code = "ERR_DATABASE"
        
        # Log full error internally
        logger.error(
            f"Database error: {str(exc)}",
            extra={
                "request_id": getattr(request.state, "request_id", "unknown"),
                "sql": str(getattr(exc, 'statement', 'unknown'))
            },
            exc_info=True
        )
        
        return create_error_response(
            request=request,
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            message=error_message,
            error_code=error_code,
            details=details,
            exc=exc
        )
    
    # Handler for JWT authentication errors
    @app.exception_handler(JWTError)
    async def jwt_exception_handler(request: Request, exc: JWTError):
        return create_error_response(
            request=request,
            status_code=status.HTTP_401_UNAUTHORIZED,
            message="Invalid or expired token",
            error_code="ERR_INVALID_TOKEN",
            exc=exc
        )
    
    # Catch-all handler for unhandled exceptions
    @app.exception_handler(Exception)
    async def global_exception_handler(request: Request, exc: Exception):
        error_id = str(uuid.uuid4())
        
        # Log full stack trace
        logger.critical(
            f"Unhandled exception: {str(exc)}",
            extra={
                "request_id": getattr(request.state, "request_id", "unknown"),
                "error_id": error_id,
                "traceback": traceback.format_exc()
            }
        )
        
        # Return generic error to client (security)
        return create_error_response(
            request=request,
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            message="An unexpected error occurred",
            error_code="ERR_INTERNAL",
            details={"error_id": error_id},  # For support reference
            exc=exc
        )


# Initialize in main.py
app = FastAPI()
setup_exception_handlers(app)
```

---

### 22.3 Structured Logging: Production-Grade Observability

Production logging requires structured formats (JSON), correlation IDs for request tracing, and appropriate log levels for different environments.

#### Structured Logging Implementation

```python
# logging_config.py - Production logging setup
import logging
import logging.config
import sys
import json
from datetime import datetime
from typing import Any, Dict
import traceback

class JSONFormatter(logging.Formatter):
    """
    JSON formatter for structured logging.
    
    Outputs logs as JSON for easy parsing by log aggregators
    (ELK, Datadog, CloudWatch, etc.)
    """
    
    def format(self, record: logging.LogRecord) -> str:
        log_data: Dict[str, Any] = {
            "timestamp": datetime.utcnow().isoformat(),
            "level": record.levelname,
            "logger": record.name,
            "message": record.getMessage(),
            "module": record.module,
            "function": record.funcName,
            "line": record.lineno,
            "thread": record.thread,
            "process": record.process,
        }
        
        # Add exception info if present
        if record.exc_info:
            log_data["exception"] = {
                "type": record.exc_info[0].__name__ if record.exc_info[0] else None,
                "message": str(record.exc_info[1]) if record.exc_info[1] else None,
                "traceback": traceback.format_exception(*record.exc_info)
            }
        
        # Add extra fields from record
        if hasattr(record, "request_id"):
            log_data["request_id"] = record.request_id
        if hasattr(record, "user_id"):
            log_data["user_id"] = record.user_id
        if hasattr(record, "duration_ms"):
            log_data["duration_ms"] = record.duration_ms
        
        # Add any extra dict attributes
        for key, value in record.__dict__.items():
            if key not in log_data and not key.startswith("_"):
                log_data[key] = value
        
        return json.dumps(log_data, default=str)


class CorrelationIdFilter(logging.Filter):
    """
    Add correlation ID to all log records.
    
    The correlation ID should be set in request state
    by middleware.
    """
    
    def filter(self, record: logging.LogRecord) -> bool:
        # Use contextvar or thread-local storage
        from contextvars import ContextVar
        
        correlation_id: ContextVar[str] = ContextVar(
            "correlation_id",
            default="unknown"
        )
        
        try:
            record.correlation_id = correlation_id.get()
        except:
            record.correlation_id = "unknown"
        
        return True


def setup_logging(log_level: str = "INFO", json_format: bool = True):
    """
    Configure application logging.
    
    Args:
        log_level: Minimum log level (DEBUG, INFO, WARNING, ERROR)
        json_format: Use JSON format (production) or console (development)
    """
    
    handlers = {
        "default": {
            "level": log_level,
            "class": "logging.StreamHandler",
            "stream": sys.stdout,
        }
    }
    
    if json_format:
        handlers["default"]["formatter"] = "json"
    else:
        handlers["default"]["formatter"] = "console"
    
    config = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {
            "json": {
                "()": JSONFormatter,
            },
            "console": {
                "format": "%(asctime)s [%(correlation_id)s] %(levelname)s: %(message)s",
                "datefmt": "%Y-%m-%d %H:%M:%S"
            }
        },
        "filters": {
            "correlation_id": {
                "()": CorrelationIdFilter
            }
        },
        "handlers": handlers,
        "loggers": {
            "": {  # Root logger
                "handlers": ["default"],
                "level": log_level,
                "propagate": False
            },
            "uvicorn": {
                "handlers": ["default"],
                "level": log_level,
                "propagate": False
            },
            "uvicorn.access": {
                "handlers": ["default"],
                "level": "WARNING",  # Reduce noise from access logs
                "propagate": False
            },
            "sqlalchemy.engine": {
                "handlers": ["default"],
                "level": "WARNING",  # Set to INFO to see queries
                "propagate": False
            }
        }
    }
    
    logging.config.dictConfig(config)


# Usage in FastAPI
from fastapi import FastAPI, Request
import contextvars

correlation_id_var = contextvars.ContextVar("correlation_id", default="unknown")

app = FastAPI()

@app.middleware("http")
async def logging_middleware(request: Request, call_next):
    """Add correlation ID to all requests."""
    # Generate or extract correlation ID
    correlation_id = request.headers.get("X-Correlation-ID", str(uuid.uuid4()))
    correlation_id_var.set(correlation_id)
    
    # Store in request state for access in endpoints
    request.state.correlation_id = correlation_id
    
    # Add to response headers
    start_time = time.time()
    
    response = await call_next(request)
    
    response.headers["X-Correlation-ID"] = correlation_id
    
    # Log request completion
    duration = (time.time() - start_time) * 1000
    
    logger.info(
        f"{request.method} {request.url.path} {response.status_code}",
        extra={
            "correlation_id": correlation_id,
            "method": request.method,
            "path": str(request.url.path),
            "status_code": response.status_code,
            "duration_ms": round(duration, 2),
            "user_agent": request.headers.get("user-agent"),
            "client_ip": request.client.host if request.client else None
        }
    )
    
    return response
```

---

### Summary

In this chapter, you implemented production-grade error handling:

1. **HTTPException**: Used standard HTTP exceptions with appropriate status codes, custom headers (Retry-After), and multi-field validation errors while preventing information leakage through generic authentication error messages.

2. **Custom Exception Handlers**: Created a hierarchy of custom exceptions (AppException, ResourceNotFoundException, etc.), registered global handlers for consistent JSON responses, and implemented specific handlers for database errors, JWT failures, and catch-all unhandled exceptions with proper logging.

3. **Structured Logging**: Implemented JSON formatting for log aggregation, correlation ID tracking across request lifecycle, configurable log levels per environment, and integration with error handlers for complete observability.

**Error Handling Best Practices:**
- Never expose internal error details or stack traces to clients
- Use consistent error response formats across all endpoints
- Include error IDs for support ticket correlation
- Log full exception details internally for debugging
- Set appropriate HTTP status codes (4xx client errors, 5xx server errors)
- Use correlation IDs to trace requests across services
- Structure logs as JSON for automated parsing

---

### What's Next?

**Chapter 23: GraphQL** will cover:
- **Introduction to GraphQL**: Understanding the query language, type system, and how it differs from REST for flexible data fetching
- **Integrating Strawberry**: Setting up Strawberry GraphQL with FastAPI for code-first schema definition, resolvers, and mutations
- **Advanced Patterns**: Implementing DataLoaders for N+1 query prevention, authentication in GraphQL context, and subscription handling for real-time updates

This next chapter expands your API design toolkit with GraphQL as an alternative to REST for specific use cases.